<a href="https://colab.research.google.com/github/taaha3244/AI-Agents/blob/main/Click_up_crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 737.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.2/339.2 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [4]:
pip install -U langchain-community tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.0.2
    Uninstalling langchain-text-splitters-0.0.2:
      Successfully uninstalled langchain-text-splitters-0.0.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.13
    Uninstalling langchain-0.1.13:
      Successfully uninstalled langchain-0.1.13
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.29
    Uninstalling langchain-community-0.0.29:
      Successfully uninstalle

- Import libraries, APIs and LLM

In [6]:
from crewai import Agent, Crew, Task

In [8]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["TAVILY_API_KEY"] =userdata.get("TAVILY_API_KEY")

In [72]:
from crewai_tools import BaseTool
import requests

class Clickup(BaseTool):
    name: str = "Fetch ClickUp goals for me"
    description: str = "The tool returns Goal name and Goal description"

    def _run(self, goal_name: str) -> dict:
        url = "https://api.clickup.com/api/v2/team/workspaceid/goal"
        query = {"include_completed": "true"}
        headers = {"Authorization": ""}

        response = requests.get(url, headers=headers, params=query)

        if response.status_code != 200:
            return {"error": f"Failed to read goals from ClickUp for {goal_name}"}

        goals = response.json()
        for goal in goals['goals']:
            if goal['name'] == goal_name:
                goal_description = goal.get('description', 'No description available')
                return {"goal_name": goal_name, "goal_description": goal_description}

        return {"error": f"No such goal: {goal_name}"}

In [79]:
b=Clickup()

In [82]:
b._run('Learn Open AI')

{'goal_name': 'Learn Open AI',
 'goal_description': 'Learn open AI assistants API\n'}

In [73]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import Tool

tool = TavilySearchResults()
tavily= Tool(
  name="Intermediate Answer",
  func=tool.run,
  description="Useful for search-based queries",
)

## Creating Agents

In [74]:
from crewai import Agent

clickup_read_tool = Clickup()

reader_agent = Agent(
    role='Goal Reader',
    goal='Read goals from ClickUp',
    verbose=True,
    memory=True,
    backstory=(
        "An agent adept at navigating ClickUp and extracting goal information."
    ),
    tools=[clickup_read_tool],
    allow_delegation=True
)

In [77]:
read_goals_task = Task(
    description=(
        "Read the goals from ClickUp for {goal_name}."
    ),
    expected_output='The goal and its description retrieved from ClickUp.',
    tools=[clickup_read_tool],
    agent=reader_agent,
)

In [16]:
researcher_agent = Agent(
    role='Researcher',
    goal='Research based on the goals for {goal_name}',
    verbose=True,
    memory=True,
    backstory=(
        "A dedicated researcher who excels at finding information on various topics."
    ),
    tools=[tavily],
    allow_delegation=False
)


In [76]:
research_task = Task(
    description=(
        "Perform research based on the provided goals for {goal_name}. "
        "Use the following goal description for context: {goal_description}."
    ),
    expected_output='A detailed report on each goal for {goal_name}, including findings related to the goal description.',
    tools=[tavily],
    agent=researcher_agent,
)

In [85]:
from crewai import Crew, Process
# Form the Crew and Kick it Off
crew = Crew(
    agents=[reader_agent, researcher_agent],
    tasks=[read_goals_task, research_task],
    process=Process.sequential  # Sequential task execution
)

# Kick off the crew with the goal name as input
result = crew.kickoff(inputs={'goal_name': 'Learn Open AI','goal_description':''})
print(result)



> Entering new CrewAgentExecutor chain...
I need to fetch the goal information for "Learn Open AI" from ClickUp.

Action: Fetch ClickUp goals for me
Action Input: {"goal_name": "Learn Open AI"} 

{'goal_name': 'Learn Open AI', 'goal_description': 'Learn open AI assistants API\n'}

Final Answer: 
Goal Name: Learn Open AI
Goal Description: Learn open AI assistants API

> Finished chain.


> Entering new CrewAgentExecutor chain...
I should start by researching the Open AI assistants API to provide a detailed report on each goal for Learn Open AI.

Action: Intermediate Answer
Action Input: {"topic": "Open AI assistants API"} 

[{'url': 'https://openai.com/api/', 'content': "Assistants API. Build AI assistants within your own applications that can leverage models, tools, and knowledge to do complex, multi-step tasks. View Assistants API docs (opens in a new window) Fine-tuning API. Customize a model's existing knowledge and behavior for a specific task via supervised fine-tuning."}, {'url

In [86]:
from IPython.display import Markdown
Markdown(result)

Goal Name: Learn Open AI
Goal Description: Learn open AI assistants API

Detailed Report:

1. Assistants API:
The Assistants API provided by OpenAI allows developers to build AI assistants within their own applications. These assistants can leverage models, tools, and knowledge to perform complex, multi-step tasks. With the Assistants API, developers can customize a model's existing knowledge and behavior for specific tasks through supervised fine-tuning. The API supports three types of tools: Code Interpreter, File Search, and Function calling.

2. OpenAI Platform:
The OpenAI Platform offers resources, tutorials, API documentation, and dynamic examples to help developers make the most out of the developer platform. This platform provides support for building AI assistants and accessing various tools for enhancing the functionality of applications.

3. Updates and Enhancements:
OpenAI is continuously improving the Assistants API and introducing new features. One notable update is the launch of an improved retrieval tool called file_search, which can now ingest up to 10,000 files per assistant - a significant increase from before. This enhancement makes file searching faster, supports parallel queries through multi-threaded searches, and includes features like enhanced reranking and query rewriting. Additionally, OpenAI is introducing vector_store objects to further enhance the capabilities of the Assistants API.

4. FAQ:
The Assistants API FAQ provides insights into the purpose and functionality of the API. It enables developers to easily create powerful AI assistants within their applications without the need to manage conversation history. Developers can access OpenAI-hosted tools like the Code Interpreter and File Search through the API. Furthermore, the API supports improved function calling for third-party tools, making it versatile and adaptable for various development requirements.

In conclusion, learning about the OpenAI assistants API involves understanding its capabilities, features, and the resources available through the OpenAI Platform. By leveraging the Assistants API, developers can create intelligent AI assistants to enhance user experiences and streamline complex tasks within their applications.